Based on the process to get grids identified in notebooks 5 and 6.

I already have created the npy images. I now need to read them in to create the train/test/validate datasets.



In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [2]:
labelsDataDir = 'C:\\Users\\JOSA\\Documents\\repos\\allImgsLabelledDB\\'
npyDataDir = 'C:\\Users\\JOSA\\Documents\\repos\\npyData\\'

allNpys = []
for file in os.listdir(npyDataDir):
    if file.endswith(".npy"):
        allNpys.append(file)


In [3]:
######################
import random
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from IPython.display import display,clear_output    
    
%matplotlib inline

# To prevent automatic figure display when execution of the cell ends
%config InlineBackend.close_figures=False 


plt.ioff()
ax=plt.gca()

out=widgets.Output()
button=widgets.Button(description='Show random square')
vbox=widgets.VBox(children=(out,button))
display(vbox)

def click(b):
    ax.clear()
    #get the name of the image and the squareIdx, needed to retrived the label from the labelled dataset
    testIdx = 0
    while testIdx == 0:
        pickImg = random.choice(allNpys)
        imgName = pickImg.split('_')[0] # + '.npy' for the data, # + '.dat' for the labels
        squareIdx = int(pickImg.split('_')[1].split('.')[0])
        # #read the dat
        #labels = np.loadtxt(os.path.join(labelsDataDir, imgName + '.dat' ))
        allLabels = []
        with open(os.path.join(labelsDataDir, imgName + '.dat' )) as f:
            allLabels.append([line.split()[0:9] for line in f])

        allLabels = allLabels[0][2:11]
        allLabels2 = []
        for i in range(0, 9):
            allLabels2 = allLabels2 + list(map(int, allLabels[i]))
    
        testIdx = allLabels2[squareIdx]
    # check labels    
    ax.imshow(np.load(os.path.join(npyDataDir, pickImg)), cmap=plt.cm.gray)
    ax.set_axis_off()
    ax.set_title(pickImg + ', ' + str(allLabels2[squareIdx]))
        
    with out:
        clear_output(wait=True)
        display(ax.figure)

button.on_click(click)
click(None)

Now we know we can correctly read in images and get the label. We now need to create the train datasets

In [4]:
x_labels = []
for i in range(0, len(allNpys)):
    pickImg = allNpys[i]
    imgName = pickImg.split('_')[0] # + '.npy' for the data, # + '.dat' for the labels
    squareIdx = int(pickImg.split('_')[1].split('.')[0])
    if os.path.isfile(os.path.join(labelsDataDir, imgName + '.dat' )):
        # #read the dat
        #labels = np.loadtxt(os.path.join(labelsDataDir, imgName + '.dat' ))
        allLabels = []
        with open(os.path.join(labelsDataDir, imgName + '.dat' )) as f:
            allLabels.append([line.split()[0:9] for line in f])

        allLabels = allLabels[0][2:11]
        allLabels2 = []
        for ii in range(0, 9):
            allLabels2 = allLabels2 + list(map(int, allLabels[ii]))
    
        x_labels.append(allLabels2[squareIdx])
        thisImg = np.load(os.path.join(npyDataDir, pickImg))
        if i == 0:
            x_train = thisImg
        else:
            x_train = np.dstack((x_train, thisImg))
                      


In [5]:
from sklearn.model_selection import train_test_split
#prepare datasets
x_train = np.transpose(x_train, (2, 0, 1))  
# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],
                                            x_train.shape[2], 1)
y_train = np.array(x_labels)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
# create testtrain/validate
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=1234)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.1, random_state = 1234)
input_shape = (30, 30, 1)

In [6]:
# train model
import tensorflow as tf
import keras
# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Creating a Sequential Model and adding the layers
model = Sequential()
model.add(Conv2D(30, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(10,activation=tf.nn.softmax))


model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=10)



model.evaluate(x_test, y_test)




Using TensorFlow backend.


Epoch 1/10
13056/13056 [==============================] - 11s 839us/step - loss: 1.4719 - accuracy: 0.6399
Epoch 2/10
13056/13056 [==============================] - 11s 819us/step - loss: 1.2808 - accuracy: 0.6412
Epoch 3/10
13056/13056 [==============================] - 10s 749us/step - loss: 0.9587 - accuracy: 0.6615
Epoch 4/10
13056/13056 [==============================] - 10s 753us/step - loss: 0.8578 - accuracy: 0.6881
Epoch 5/10
13056/13056 [==============================] - 10s 778us/step - loss: 0.8139 - accuracy: 0.7041
Epoch 6/10
13056/13056 [==============================] - 10s 800us/step - loss: 0.7723 - accuracy: 0.7206
Epoch 7/10
13056/13056 [==============================] - 10s 759us/step - loss: 0.7408 - accuracy: 0.7351
Epoch 8/10
13056/13056 [==============================] - 10s 771us/step - loss: 0.7097 - accuracy: 0.7479
Epoch 9/10
13056/13056 [==============================] - 10s 754us/step - loss: 0.6803 - accuracy: 0.7571
Epoch 10/10
1612/1612 [==============

[0.6167320752927742, 0.7990074157714844]

In [7]:
%matplotlib inline

# To prevent automatic figure display when execution of the cell ends
%config InlineBackend.close_figures=False 


plt.ioff()
ax=plt.gca()

out=widgets.Output()
button=widgets.Button(description='Show random square')
vbox=widgets.VBox(children=(out,button))
display(vbox)

def click(b):
    ax.clear()
    image_index = random.choice(range(0, x_test.shape[0]))
    #get the name of the image and the squareIdx, needed to retrived the label from the labelled dataset
    pred = model.predict(x_test[image_index].reshape(1, x_test[image_index].shape[0], x_test[image_index].shape[1], 1))
    
    # check labels    
    ax.imshow(x_test[image_index].reshape(30, 30), cmap=plt.cm.gray)
    ax.set_axis_off()
    ax.set_title(str(pred.argmax()))
        
    with out:
        clear_output(wait=True)
        display(ax.figure)

button.on_click(click)
click(None)

In [34]:
from keras.layers import Dense, Activation, Dropout, Convolution2D, Flatten, MaxPooling2D, Reshape, InputLayer
# define vars
input_shape = (30,30,1)
#input_reshape = (30, 30, 1)

conv_num_filters = 5
conv_filter_size = 5

pool_size = (2, 2)

hidden_num_units = 128
output_num_units = 10

epochs = 50 # 30 should be enough to plateu performance
batch_size = 128

######
# model = Sequential()
# model.add(Conv2D(30, kernel_size=(3,3), input_shape=input_shape))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
# model.add(Dense(128, activation=tf.nn.relu))
# model.add(Dropout(0.2))
# model.add(Dense(10,activation=tf.nn.softmax))
##############
model = Sequential()#[InputLayer(input_shape=input_shape),
model.add(Conv2D(30, (3, 3), input_shape=input_shape))#,
model.add(MaxPooling2D(pool_size=pool_size))#,
model.add(Conv2D(30, (3, 3)))#,
model.add(MaxPooling2D(pool_size=pool_size))#,
model.add(Conv2D(30, (3, 3)))#,
model.add(Flatten())#,
model.add(Dense(units=hidden_num_units, activation='relu'))#,
model.add(Dense(units=output_num_units, input_dim=hidden_num_units, activation='softmax'))#,
#])
model.compile(loss ="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

model.fit(x=x_train, y=y_train, epochs=epochs, batch_size=batch_size,
          validation_data=(x_test, y_test))


Train on 13056 samples, validate on 1612 samples
Epoch 1/50
13056/13056 [==============================] - 9s 673us/step - loss: 1.5185 - accuracy: 0.6350 - val_loss: 1.4348 - val_accuracy: 0.6433
Epoch 2/50
13056/13056 [==============================] - 9s 670us/step - loss: 1.4418 - accuracy: 0.6412 - val_loss: 1.4266 - val_accuracy: 0.6433
Epoch 3/50
13056/13056 [==============================] - 8s 631us/step - loss: 1.4196 - accuracy: 0.6412 - val_loss: 1.3734 - val_accuracy: 0.6433
Epoch 4/50
13056/13056 [==============================] - 8s 639us/step - loss: 1.2595 - accuracy: 0.6439 - val_loss: 1.0150 - val_accuracy: 0.6737
Epoch 5/50
13056/13056 [==============================] - 8s 624us/step - loss: 0.8847 - accuracy: 0.6952 - val_loss: 0.8181 - val_accuracy: 0.7103
Epoch 6/50
13056/13056 [==============================] - 8s 618us/step - loss: 0.7767 - accuracy: 0.7291 - val_loss: 0.7299 - val_accuracy: 0.7550
Epoch 7/50
13056/13056 [==============================] - 8s 61

In [35]:
%matplotlib inline

# To prevent automatic figure display when execution of the cell ends
%config InlineBackend.close_figures=False 


plt.ioff()
ax=plt.gca()

out=widgets.Output()
button=widgets.Button(description='Show random square')
vbox=widgets.VBox(children=(out,button))
display(vbox)

def click(b):
    ax.clear()
    image_index = random.choice(range(0, x_test.shape[0]))
    #get the name of the image and the squareIdx, needed to retrived the label from the labelled dataset
    pred = model.predict(x_test[image_index].reshape(1, x_test[image_index].shape[0], x_test[image_index].shape[1], 1))
    
    # check labels    
    ax.imshow(x_test[image_index].reshape(30, 30), cmap=plt.cm.gray)
    ax.set_axis_off()
    ax.set_title(str(pred.argmax()))
        
    with out:
        clear_output(wait=True)
        display(ax.figure)

button.on_click(click)
click(None)

# Save model

In [37]:
model.save("numbersModel.h5")
print("Saved model to disk")

Saved model to disk
